In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# plotting stuff
from pandas.plotting import lag_plot
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
colorMap = sns.light_palette("blue", as_cmap=True)

# system
import warnings
warnings.filterwarnings('ignore')
# for the image import

from IPython.display import Image
# garbage collector to keep RAM in check
import gc 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
resp = pd.read_csv('/kaggle/input/jane-street-real-time-market-data-forecasting/responders.csv')
resp.tail()

In [ ]:
#Features file, index_col=0 so the index is feature column

feat = pd.read_csv('/kaggle/input/jane-street-real-time-market-data-forecasting/features.csv', index_col=0)
feat.tail()

In [ ]:
#Features file, index_col=0 so the index is feature column

feat = pd.read_csv('/kaggle/input/jane-street-real-time-market-data-forecasting/features.csv', index_col=0)
feat.tail()

In [ ]:
#By Safadoost
#encode the boolean  True/False to plot a heatmap after

#By Safadoost
#encode the boolean  True/False to plot a heatmap after

categorical_columns = feat[:]
# Create a new column for each unique value in the categorical columns.
df_categorical = pd.get_dummies(
    data = feat ,
    prefix = 'OHE' ,  # One hot encoding
    prefix_sep = '_' ,
    columns = categorical_columns ,
    #drop_first = True ,
    dtype = 'int8'
)


In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df_categorical.corr(),annot=True);

In [ ]:
#Read One parquet file. Obviously, it's big.

train_data = pd.read_parquet("/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=4/part-0.parquet")
train_data.tail()

In [ ]:
#By Carl McBride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train_data['responder_0']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_ylabel ("Cumulative responder 0", fontsize=18);
balance.plot(lw=3);
del balance
gc.collect();

In [ ]:
#By Carl McBride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

fig, ax = plt.subplots(figsize=(15, 5))
balance= pd.Series(train_data['responder_0']).cumsum()
resp_1= pd.Series(train_data['responder_1']).cumsum()
resp_2= pd.Series(train_data['responder_2']).cumsum()
resp_3= pd.Series(train_data['responder_3']).cumsum()
resp_4= pd.Series(train_data['responder_4']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cumulative resp and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
balance.plot(lw=3)
resp_1.plot(lw=3)
resp_2.plot(lw=3)
resp_3.plot(lw=3)
resp_4.plot(lw=3)
plt.legend(loc="upper left");
del resp_1
del resp_2
del resp_3
del resp_4
gc.collect();

In [ ]:
#By Carl McBride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['responder_0'], 
             bins=30, #Original was 3000 bins
             kde_kws={"clip":(-0.05,0.05)}, 
             hist_kws={"range":(-0.05,0.05)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the resp values", size=14)
plt.show();
gc.collect();


In [ ]:
min_resp = train_data['responder_0'].min()
print('The minimum value for resp is: %.5f' % min_resp)
max_resp = train_data['responder_0'].max()
print('The maximum value for resp is:  %.5f' % max_resp)

In [ ]:
min_resp = train_data['responder_0'].min()
print('The minimum value for resp is: %.5f' % min_resp)
max_resp = train_data['responder_0'].max()
print('The maximum value for resp is:  %.5f' % max_resp)

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

#I don't know why I had to define values again when I tried to change the numbers
#Anyway this line is in every histogram
values = np.array([rec.get_height() for rec in ax.patches])

from scipy.optimize import curve_fit
# the values
x = list(range(len(values)))
x = [((i)-15)/30 for i in x]
y = values

def Lorentzian(x, x0, gamma, A):
    return A * gamma**2/(gamma**2+( x - x0 )**2)

# seed guess
initial_guess=(0, 0.001, 30)

# the fit
parameters,covariance=curve_fit(Lorentzian,x,y,initial_guess)
sigma=np.sqrt(np.diag(covariance))

# and plot
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['responder_0'], 
             bins=30, 
             kde_kws={"clip":(-0.05,0.05)}, 
             hist_kws={"range":(-0.05,0.05)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
#norm = plt.Normalize(values.min(), values.max())
#colors = plt.cm.jet(norm(values))
#for rec, col in zip(ax.patches, colors):
#    rec.set_color(col)
plt.xlabel("Histogram of the responder_0 values", size=14)
plt.plot(x,Lorentzian(x,*parameters),'--',color='black',lw=3)
plt.show();
del values
gc.collect();


In [ ]:
percent_zeros = (100/train_data.shape[0])*((train_data.weight.values == 0).sum())
print('Percentage of zero weights is: %i' % percent_zeros +"%")

In [ ]:
min_weight = train_data['weight'].min()
print('The minimum weight is: %.2f' % min_weight)

In [ ]:
max_weight = train_data['weight'].max()
print('The maximum weight was: %.2f' % max_weight)

In [ ]:
train_data[train_data['weight']==train_data['weight'].max()]

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['weight'], 
             bins=140, #Original is 1400
             kde_kws={"clip":(0.001,1.4)}, 
             hist_kws={"range":(0.001,1.4)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of non-zero weights", size=14)
plt.show();
del values
gc.collect();

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

train_data_nonZero = train_data.query('weight > 0').reset_index(drop = True)
plt.figure(figsize = (10,4))
ax = sns.distplot(np.log(train_data_nonZero['weight']), 
             bins=100, #Original was 1000
             kde_kws={"clip":(-4,5)}, 
             hist_kws={"range":(-4,5)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the logarithm of the non-zero weights", size=14)
plt.show();
gc.collect();


In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

values = np.array([rec.get_height() for rec in ax.patches])

from scipy.optimize import curve_fit
# the values
x = list(range(len(values)))
x = [(i/11)-4 for i in x]  #Original was 110
y = values

# define a Gaussian function
def Gaussian(x,mu,sigma,A):
    return A*np.exp(-0.5 * ((x-mu)/sigma)**2)

def bimodal(x,mu_1,sigma_1,A_1,mu_2,sigma_2,A_2):
    return Gaussian(x,mu_1,sigma_1,A_1) + Gaussian(x,mu_2,sigma_2,A_2)

# seed guess
initial_guess=(1, 1 , 1,    1, 1, 1)

# the fit
parameters,covariance=curve_fit(bimodal,x,y,initial_guess)
sigma=np.sqrt(np.diag(covariance))

# the plot
plt.figure(figsize = (10,4))
ax = sns.distplot(np.log(train_data_nonZero['weight']), 
             bins=100,     #Original was 1000
             kde_kws={"clip":(-4,5)}, 
             hist_kws={"range":(-4,5)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the logarithm of the non-zero weights", size=14)
# plot gaussian #1
plt.plot(x,Gaussian(x,parameters[0],parameters[1],parameters[2]),':',color='black',lw=2,label='Gaussian #1', alpha=0.8)
# plot gaussian #2
plt.plot(x,Gaussian(x,parameters[3],parameters[4],parameters[5]),'--',color='black',lw=2,label='Gaussian #2', alpha=0.8)
# plot the two gaussians together
plt.plot(x,bimodal(x,*parameters),color='black',lw=2, alpha=0.7)
plt.legend(loc="upper left");
plt.show();
del values
gc.collect();


In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

train_data['feature_00']   = train_data['weight']*train_data['responder_0']
train_data['feature_01'] = train_data['weight']*train_data['responder_1']
train_data['feature_02'] = train_data['weight']*train_data['responder_2']
train_data['feature_03'] = train_data['weight']*train_data['responder_3']
train_data['feature_04'] = train_data['weight']*train_data['responder_4']

fig, ax = plt.subplots(figsize=(15, 5))
resp    = pd.Series(1+(train_data.groupby('date_id')['feature_00'].mean())).cumprod()
resp_1  = pd.Series(1+(train_data.groupby('date_id')['feature_01'].mean())).cumprod()
resp_2  = pd.Series(1+(train_data.groupby('date_id')['feature_02'].mean())).cumprod()
resp_3  = pd.Series(1+(train_data.groupby('date_id')['feature_03'].mean())).cumprod()
resp_4  = pd.Series(1+(train_data.groupby('date_id')['feature_04'].mean())).cumprod()
ax.set_xlabel ("Day", fontsize=18)
ax.set_title ("Cumulative daily return for responder and time horizons 1, 2, 3, and 4 (500 days)", fontsize=18)
resp.plot(lw=3, label='responder_0 x weight')
resp_1.plot(lw=3, label='responder_1 x weight')
resp_2.plot(lw=3, label='responder_2 x weight')
resp_3.plot(lw=3, label='responder_3 x weight')
resp_4.plot(lw=3, label='responder_4 x weight')
# day 85 marker
ax.axvline(x=85, linestyle='--', alpha=0.3, c='red', lw=1)
ax.axvspan(0, 85 , color=sns.xkcd_rgb['grey'], alpha=0.1)
plt.legend(loc="lower left");


In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

train_data_no_0 = train_data.query('weight > 0').reset_index(drop = True)
train_data_no_0['wAbsResp'] = train_data_no_0['weight'] * (train_data_no_0['responder_0'])
#plot
plt.figure(figsize = (12,5))
ax = sns.distplot(train_data_no_0['wAbsResp'], 
             bins=150,  #Original was 1500
             kde_kws={"clip":(-0.02,0.02)}, 
             hist_kws={"range":(-0.02,0.02)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of the weights * resp", size=14)
plt.show();

In [ ]:
train_data['feature_00'].value_counts()

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

fig, ax = plt.subplots(figsize=(15, 4))
feature_0 = pd.Series(train_data['feature_00']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_ylabel ("feature_00 (cumulative)", fontsize=18);
feature_0.plot(lw=3);

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2,figsize=(20,10))

ax1.plot((pd.Series(train_data['feature_01']).cumsum()), lw=3, color='red')
ax1.set_title ("Noisy", fontsize=22);
ax1.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax1.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax1.set_xlim(xmin=0)
ax1.set_ylabel ("feature_01", fontsize=18);

ax2.plot((pd.Series(train_data['feature_03']).cumsum()), lw=3, color='green')
ax2.set_title ("Negative", fontsize=22);
ax2.axvline(x=514052, linestyle='--', alpha=0.3, c='red', lw=2)
ax2.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax2.set_xlim(xmin=0)
ax2.set_ylabel ("feature_03", fontsize=18);

ax3.plot((pd.Series(train_data['feature_55']).cumsum()), lw=3, color='darkorange')
ax3.set_title ("Hybryd (Tag 21)", fontsize=22);
ax3.set_xlabel ("Trade", fontsize=18)
ax3.axvline(x=514052, linestyle='--', alpha=0.3, c='green', lw=2)
ax3.axvspan(0, 514052 , color=sns.xkcd_rgb['grey'], alpha=0.1)
ax3.set_xlim(xmin=0)
ax3.set_ylabel ("feature_55", fontsize=18);

ax4.plot((pd.Series(train_data['feature_73']).cumsum()), lw=3, color='blue')
ax4.set_title ("Hybryd", fontsize=22)
ax4.set_xlabel ("Trade", fontsize=18)
ax4.set_ylabel ("feature_73", fontsize=18);
gc.collect();

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

fig, ax = plt.subplots(figsize=(15, 5))
feature_60= pd.Series(train_data['feature_60']).cumsum()
feature_61= pd.Series(train_data['feature_61']).cumsum()
feature_62= pd.Series(train_data['feature_62']).cumsum()
feature_63= pd.Series(train_data['feature_63']).cumsum()
feature_64= pd.Series(train_data['feature_64']).cumsum()
feature_65= pd.Series(train_data['feature_65']).cumsum()
feature_66= pd.Series(train_data['feature_66']).cumsum()
feature_67= pd.Series(train_data['feature_67']).cumsum()
feature_68= pd.Series(train_data['feature_68']).cumsum()
#feature_69= pd.Series(train_data['feature_69']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cumulative plot for feature_60 ... feature_68.", fontsize=18)
feature_60.plot(lw=3)
feature_61.plot(lw=3)
feature_62.plot(lw=3)
feature_63.plot(lw=3)
feature_64.plot(lw=3)
feature_65.plot(lw=3)
feature_66.plot(lw=3)
feature_67.plot(lw=3)
feature_68.plot(lw=3)
#feature_69.plot(lw=3)
plt.legend(loc="upper left");
del feature_60, feature_61, feature_62, feature_63, feature_64, feature_65, feature_66 ,feature_67, feature_68
gc.collect();

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

sns.set_palette("bright")

fig, axes = plt.subplots(2,2,figsize=(8,8))

sns.distplot(train_data[['feature_60']], hist=True, bins=200,  ax=axes[0,0])
sns.distplot(train_data[['feature_61']], hist=True, bins=200,  ax=axes[0,0])
axes[0,0].set_title ("features 60 and 61", fontsize=18)
axes[0,0].legend(labels=['60', '61'])

sns.distplot(train_data[['feature_62']], hist=True,  bins=200, ax=axes[0,1])
sns.distplot(train_data[['feature_63']], hist=True,  bins=200, ax=axes[0,1])
axes[0,1].set_title ("features 62 and 63", fontsize=18)
axes[0,1].legend(labels=['62', '63'])

sns.distplot(train_data[['feature_65']], hist=True,  bins=200, ax=axes[1,0])
sns.distplot(train_data[['feature_66']], hist=True,  bins=200, ax=axes[1,0])
axes[1,0].set_title ("features 65 and 66", fontsize=18)
axes[1,0].legend(labels=['65', '66'])


sns.distplot(train_data[['feature_67']], hist=True,  bins=200, ax=axes[1,1])
sns.distplot(train_data[['feature_68']], hist=True,  bins=200, ax=axes[1,1])
axes[1,1].set_title ("features 67 and 68", fontsize=18)
axes[1,1].legend(labels=['67', '68'])

plt.show();
gc.collect();

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

plt.figure(figsize = (12,5))
ax = sns.distplot(train_data['feature_64'], 
             bins=120, 
             kde_kws={"clip":(-6,6)}, 
             hist_kws={"range":(-6,6)},
             color='darkcyan', 
             kde=False);
values = np.array([rec.get_height() for rec in ax.patches])
norm = plt.Normalize(values.min(), values.max())
colors = plt.cm.jet(norm(values))
for rec, col in zip(ax.patches, colors):
    rec.set_color(col)
plt.xlabel("Histogram of feature_64", size=14)
plt.show();
del values
gc.collect();

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

fig, ax = plt.subplots(figsize=(15, 4))
ax.scatter(train_data_nonZero.weight, train_data_nonZero.feature_51, s=0.1, color='b')
ax.set_xlabel('weight')
ax.set_ylabel('feature_51')
plt.show();

In [ ]:
#By Carl Mcbride Ellis https://www.kaggle.com/code/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance/notebook

fig, ax = plt.subplots(figsize=(15, 5))
feature_55= pd.Series(train_data['feature_55']).cumsum()
feature_56= pd.Series(train_data['feature_56']).cumsum()
feature_57= pd.Series(train_data['feature_57']).cumsum()
feature_58= pd.Series(train_data['feature_58']).cumsum()
feature_59= pd.Series(train_data['feature_59']).cumsum()
ax.set_xlabel ("Trade", fontsize=18)
ax.set_title ("Cumulative plot for the 'Tag 21' features (55-59)", fontsize=18)
ax.axvline(x=514052, linestyle='--', alpha=0.3, c='black', lw=1)
ax.axvspan(0,  514052, color=sns.xkcd_rgb['grey'], alpha=0.1)
feature_55.plot(lw=3)
feature_56.plot(lw=3)
feature_57.plot(lw=3)
feature_58.plot(lw=3)
feature_59.plot(lw=3)
plt.legend(loc="upper left");
gc.collect();


In [ ]:
import pandas as pd
from itertools import combinations, product

data = np.array([
    [1, 0, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 0, 1, 1],
    [0, 1, 0, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0]
])

df = pd.DataFrame(data.T, columns=[f'r{i}' for i in range(9)])

candidates = [1, 2, 4, 5, 7, 8]  # r1, r2, r4, r5, r7, r8

expressions = []

for i in range(1, len(candidates) + 1):
    for combo in combinations(candidates, i):
        for signs in product([-1, 1], repeat=len(combo)):
            expr = "df['r6'] == df['r0']"
            for sign, row in zip(signs, combo):
                expr += f" + {sign} * df['r{row}']"  # df 中的列名
            expressions.append(expr)

results = []
for expr in expressions:
    result = eval(expr)
    if all(result):
        print(expr)

In [ ]:
candidates = [1, 2, 4, 5, 7, 8]  # r1, r2, r4, r5, r7, r8

expressions = []

for i in range(1, len(candidates) + 1):
    for combo in combinations(candidates, i):
        for signs in product([-1, 1], repeat=len(combo)):
            expr = "df['r6'] == df['r3']"
            for sign, row in zip(signs, combo):
                expr += f" + {sign} * df['r{row}']"  # df 中的列名
            expressions.append(expr)

results = []
for expr in expressions:
    result = eval(expr)
    if all(result):
        print(expr)

In [ ]:
features_data = pd.read_csv(r'/kaggle/input/jane-street-real-time-market-data-forecasting/features.csv')
responders_data = pd.read_csv(r'/kaggle/input/jane-street-real-time-market-data-forecasting/responders.csv')
sample_data = pd.read_csv(r'/kaggle/input/jane-street-real-time-market-data-forecasting/sample_submission.csv')

In [ ]:
print("features_data :", features_data.shape)
print("responders_data :", responders_data.shape)
print("sample_data :", sample_data.shape)

In [ ]:
features_data.head()

In [ ]:
features_data.isnull().sum().sort_values(ascending=False)

In [ ]:
responders_data.head()

In [ ]:
# Exclude a specific column, for example 'category2'
columns_to_plot = responders_data.columns.drop('responder')

# Loop through each categorical column and create a pie chart
for col in columns_to_plot:
    plt.figure(figsize=(6, 6))
    responders_data[col].value_counts().plot.pie(autopct='%1.1f%%', figsize=(5, 5))
    plt.title(f'Distribution of {col}')  # Add title for each pie chart
    plt.ylabel('')  # Removes the default y-label
    plt.show()

In [ ]:
sample_data.head()

In [ ]:
import polars as pl

In [ ]:
train_data = pl.read_parquet("/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet")
train_data.head()

In [ ]:
test_data = pl.read_parquet("/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet")
test_data.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.model_selection import train_test_split
X = train_data.drop(['partition_id','responder_0','responder_1','responder_2','responder_3','responder_4','responder_5','responder_6','responder_7','responder_8'])
y = train_data['responder_6']
test = test_data.drop(['row_id','is_scored'])

In [ ]:
X.shape, y.shape, test.shape

In [ ]:
print(type(X))
print(type(X.sample))

In [ ]:
print(type(y))

In [ ]:
import polars as pl

# Step 1: Sample 10% of the data in Polars
X_sample = X.sample(fraction=0.1, seed=42)

# Step 2: Get the row indices of the sampled rows
sample_indices = X_sample.get_column('date_id').to_numpy().astype(int)

# Step 3: Use these indices to sample y
y_sample = y[sample_indices]

# Display the results
print(X_sample)
print(y_sample)

In [ ]:
# Convert Polars DataFrame to Pandas DataFrame for sklearn compatibility
X_sample_pd = X_sample.to_pandas()
y_sample_pd = y_sample.to_pandas()

In [ ]:
# Step 4: Define categorical and numerical features
numeric_features = X_sample_pd.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = X_sample_pd.select_dtypes(exclude=[np.number]).columns.tolist()

In [ ]:
# Step 5: Define preprocessors for numeric and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('scaler', StandardScaler())                  # Scale numerical features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))      # One-hot encode categorical features
])

In [ ]:
# Step 6: Combine preprocessors into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# Step 7: Define the pipeline with preprocessing and LGBMRegressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LGBMRegressor(random_state=42))  # LightGBM Regressor
])

In [ ]:
# Step 8: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sample_pd, y_sample_pd, test_size=0.2, random_state=42)

In [ ]:
# Step 9: Fit the pipeline
pipeline.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# Get predictions
predictions = pipeline.predict(X_test)
# Display metrics
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('FloodProbability Predictions')
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
# Assuming you have a Polars DataFrame `test` for making predictions
# Convert the test DataFrame from Polars to Pandas
X_test_pd = test.to_pandas()

# Ensure your test DataFrame has the same structure as your training data
# Use the pipeline to predict
pred = pipeline.predict(X_test_pd)

# Print the predictions
#print(pred)

In [ ]:
submission = pd.DataFrame({'row_id': sample_data.row_id, 'responder_6': pred})
#print(submission.head())
submission.to_csv('submission.csv', index=False)